### 啊。。。这个就是加了好几层的神经网络，前面几层都是tanh，最后是sigmoid
### 怕遗漏一些细节，打一遍看看
### 反向传播的式子比较复杂，如果需要应用，查书是比较靠谱的选择，虽然求导不复杂，就怕写着写着就错了

In [1]:
import numpy as np
# 课程里给了一些sigmoid tanh计算的函数，都不难，就不导入了，直接记一些思路

In [2]:
# 2层的神经网络
# 老规矩先初始化一波
def initialize_parameters(n_x,n_h,n_y):
    W1 = np.random.randn(n_h,n_x)*0.01  #这里选0.01其实是防止你的input太大/太小，我们之后的斜率约等于0，迭代速度太慢
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    
    parameters = {
        'W1':W1,
        'b1':b1,
        'W2':W2,
        'b2':b2
    }
    return parameters

In [3]:
def initialize_parameters_deep(layer_dims):
    # 这个函数其实是上面函数的一般化，layer_dims是一个list，记录了每层有几个神经元，包括输入层和输出层
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1,L):
        parameters['W'+str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b'+str(l)] = np.zeros((layer_dims[l],1))
        
    return parameters

In [4]:
# 正向传播，单层
def linear_forward(A, W, b):
    Z = np.dot(W,A) + b
    cache = (A, W, b)
    return Z, cache

In [5]:
# 适用于一个神经网络有不同的activation function
def linear_activation_forward(A_prev, W, b, activation):
    if activation == 'sigmoid':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        
    elif activation == 'relu':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    cache = (linear_cache, activation_cache)
    return A, cache

In [7]:
# 整合
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters)//2  #获得神经网络的层数，嘿嘿
    
    for l in range(1,L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W'+str(l)],parameters['b'+str(l)],'relu')
        caches.append(cache)
        
    AL, cache = linear_activation_forward(A, parameters['W'+str(L)], parameters['b'+str(L)],'sigmoid')
    caches.append(cache)
    return AL, caches # AL是最后一个A

In [8]:
# 计算误差
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -1/m*np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    cost = np.squeeze(cost)
    return cost

In [9]:
# 反向传播
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = 1/m*np.dot(dZ,np.transpose(A_prev))
    db = 1/m*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(np.transpose(W),dZ)
    return dA_prev, dW, db

In [10]:
# 同样的，应用于有不同activition function的层
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation =='relu':
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

In [11]:
# 整合
def L_model_backward(Al, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = -(np.divide(Y, AL)-np.divide(1-Y,1-AL))
    
    current_cache = caches[L-1]
    grads['dA'+str(L-1)],grads['dW'+str(L)],grads['db'+str(L)] = linear_activation_backward(dAL, current_cache, 'sigmoid')
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads['dA'+str(l+1)], current_cache, 'relu')
        grads['dA'+str(l)] = dA_prev_temp
        grads['dW'+str(l+1)] = dW_temp
        grads['db'+str(l+1)] = db_temp
    return grads

In [12]:
# 更新参数
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters)//2
    for l in range(L):
        parameters['W'+str(l+1)] = parameters['W'+str(l+1)] - learning_rate*grads['dW'+str(l+1)]
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)] - learning_rate*grads['db'+str(l+1)]
    return parameters

### 应用的话，其实就是上述函数的整合，下面是两层和多层模型的应用
### 可以用来识别猫脸，算是一个很简单的神经网络应用
### 还是觉得蛮神奇的，啥都没干，搞搞数学就识别出来了，优秀！

In [1]:
def two_layer_model(X, Y, layer_dims, learning_rate=0.0075, num_iterations = 3000,print_cost=False):
    grads = {}
    costs = []
    m = X.shape[1]
    (n_x,n_h,n_y) = layers_dims
    
    parameters = initialize_parameters(n_x,n_h,n_y)
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    for i in range(0, um_iterations):
        A1,cache1 = linear_activation_forward(X, W1, b1, 'relu')
        A2, cache2 = linear_activation_forward(A1, W2, b2, 'sigmoid')
        
        cost = compute_cost(A2, Y)
        
        dA2 = -(np.divide(Y, A2) - np.divide(1-Y,1-A2))
        
        dA1, dW2, db2 = linear_activation_backward(dA2, cache2, 'sigmoid')
        dA0, dW1, db1 = linear_activation_backward(dA1, cache, 'relu')
        
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2
        
        parameters = update_parameters(parameters, grads, learning_rate)
        
        W1 = parameters['W1']
        b1 = parameters['b1']
        W2 = parameters['W2']
        b2 = parameters['b2']
        
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    return parameters

In [ ]:
def L_layer_model(X, Y, layer_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost = False):
    # a...就快打完了
    costs = []
    
    parameters = initialize_parameters_deep(layer_dims)
    
    for i in range(0, num_iterations):
        AL, caches = L_model_forward(X, parameters)
        
        cost = compute_cost(AL, Y)
        grads = L_model_backward(Al, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if print_cost and i% 100 == 0:
            costs.append(cost)
            
    return parameters